In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/bit_conference/

/content/gdrive/.shortcut-targets-by-id/1YDrmXvwQeDTF3AVegVo_-qlULY2-1-qE/bit_conference


In [ ]:
!pip install fasttext nrclex datasets contractions

In [ ]:
# Standard data processing libraries
import pandas as pd
import numpy as np

# Libraries for NLP
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt_tab')

# Language Detection Library
import fasttext as ft

# Deep learning libraries
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Embedding, Bidirectional

# Libraries for visualization
import plotly.express as px
import plotly.io as pio

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Setting template for all plots
pio.templates.default = "simple_white"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df = pd.read_csv('song_lyrics/song_lyrics.csv')
df

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
5134851,Ocean,pop,Effemar,2022,3,{},[Verse 1]\nDance for me now\nKeeping yourself ...,7882842,en,en,en
5134852,64 Bars,rap,Rapido,2022,4,{},"[Intro]\n\nJa, ja\n\n[Part 1]\n\nR-A-H, Merhab...",7882843,de,de,de
5134853,Raise Our Hands,pop,"Culture Code, Pag & Mylo",2016,3,"{Elex,""Culture Code / Pag & Mylo""}",[Verse 1]\nHere our purpose feels alive\nWe ar...,7882845,en,en,en
5134854,CEO,rap,Antropolita,2022,5,{},Jestem CEO w tym\nTo jara twoją bitch\nNikt na...,7882846,pl,pl,pl


In [ ]:
df_en = df[(df['language_cld3']=='en') & (df['language_ft']=='en')]

In [ ]:
import string
import pandas as pd
import multiprocessing
from tqdm import tqdm
from joblib import Parallel, delayed
tqdm.pandas()

def text_cleansing(text):
    if pd.isna(text):
        return ""
    text = pd.Series([text]).str.replace(r"[\(\[].*?[\)\]]", '', regex=True).values[0]
    text = text.replace("\n", ' ')
    text = text.lower()
    text = pd.Series([text]).str.replace(f"[{string.punctuation}]", '', regex=True).values[0]
    text = pd.Series([text]).str.replace(r"\s+", ' ', regex=True).str.strip().values[0]
    return text

def parallel_apply(data, func, num_cores=None):
    """병렬 처리를 통해 `func`을 `data`의 각 요소에 적용"""
    if num_cores is None:
        num_cores = multiprocessing.cpu_count()  # 사용 가능한 CPU 코어 수 확인
    print(f" 병렬 처리 시작... (사용할 코어 수: {num_cores})")
    results = Parallel(n_jobs=num_cores)(delayed(func)(text) for text in tqdm(data, desc="Processing"))
    return results

# 병렬 처리 적용
df_en['rem_sp_char'] = parallel_apply(df_en['lyrics'], text_cleansing)


 병렬 처리 시작... (사용할 코어 수: 12)


Processing: 100%|██████████| 3374198/3374198 [06:50<00:00, 8219.72it/s]
<ipython-input-9-6be303640672>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en['rem_sp_char'] = parallel_apply(df_en['lyrics'], text_cleansing)


In [ ]:
import contractions

def expand_contractions(text):
    return contractions.fix(text)

def parallel_apply(data, func, num_cores=None):
    if num_cores is None:
        num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(func)(text) for text in tqdm(data, desc="Processing"))
    return results

df_en["lyrics_formal"] = parallel_apply(df_en["rem_sp_char"], expand_contractions)

Processing: 100%|██████████| 3374198/3374198 [01:10<00:00, 48090.48it/s]
<ipython-input-25-5184b9987fff>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["lyrics_formal"] = parallel_apply(df_en["rem_sp_char"], expand_contractions)


In [ ]:
df_en = df_en.drop(columns=['features', 'lyrics', 'id', 'language', 'rem_sp_char'])

In [ ]:
taboo_words = [
    # 감탄사 및 의미 없는 단어
    "yeah", "ya", "yah", "nah", "na", "wan", "uh", "uhhuh", "mm", "mmm", "oooh", "ooh", "oh", "aha", "whoa",
    "hmm", "huh", "woo", "woah", "hahaha", "haha", "ha", "tsk", "yikes", "wow", "yup", "nope", "ayy", "aye",
    "whoo", "yoo", "damn", "gah", "eww", "ew", "tsktsk", "ugh", "meh", "nahh", "ahh", "aah", "sheesh",

    # 의성어 및 반복적 표현
    "lala", "lalala", "la", "da", "dada", "di", "didi", "dum", "dumdum", "boom", "bam", "pow",
    "bada", "bambam", "shh", "beep", "doot", "whoosh", "zoom", "zing", "clang", "blam", "ba",
    "tada", "ta", "sha", "sho", "bling", "bibidi", "shoop", "doo", "skrrt", "brr", "grr",

    # 비속어 및 부적절한 표현 (필요에 따라 조정 가능)
    "nigga", "niggas", "bitch", "hoe", "thot", "fuck", "shit", "damn", "ass", "cunt", "dick", "pussy", "motherfucker",
    "bullshit", "cock", "whore", "fucked", "fuckin", "fucka", "fck", "biatch", "mofo", "mf", "slut", "crap", "bastard",
    "douche", "dumbass", "retard", "jackass", "dipshit", "shitface", "pendejo", "puta", "chode", "twat", "wanker", "arse",

    # 특정한 반복적인 단어들 (가사에서 자주 등장하지만 의미 없는 단어들)
    "shawty", "lil", "ol", "homie", "bruh", "bro", "sis", "fam", "gang", "tonka", "drip", "flex", "gucci", "blang",
    "ice", "racks", "stacks", "bands", "whip", "dope", "lit", "turnt", "smh", "jk", "idk", "omg", "lol", "rofl", "xd",
    "holla", "bling", "grind", "ballin", "savage", "thug", "pimp", "hunnid", "trap", "yolo", "finesse", "clout",

    # 인터넷 및 소셜 미디어 단어 (필요에 따라 조정 가능)
    "meme", "selfie", "vibe", "vibin", "rizz", "cap", "no cap", "bet", "drip", "sus", "goat", "based", "cringe",
    "lmao", "lmfao", "wtf", "wth", "tf", "tbh", "fml", "irl", "brb", "btw", "stfu", "gtfo", "nvm", "smh",

    # 특정한 소리 및 반복적인 후렴구 단어들
    "yaka", "shoorah", "ook", "mmmmmm", "bidi", "bidibambambam", "oohooh", "yah", "yuh", "uhh",
    "skrrt", "brrr", "grrah", "bow", "bop", "bopbop", "choo", "chee", "skibidi", "womp", "wub",
]


In [ ]:
def remove_stopwords_single(lyric):
    ignore = set(taboo_words)
    tokens = nltk.word_tokenize(lyric)
    filtered_tokens = [token for token in tokens if token.lower() not in ignore and len(token) > 1]
    return " ".join(filtered_tokens)

def parallel_remove_stopwords(data):
    num_cores = multiprocessing.cpu_count()
    return Parallel(n_jobs=num_cores)(
        delayed(remove_stopwords_single)(lyric) for lyric in tqdm(data, desc="Processing")
    )

df_en['lyrics_clean'] = parallel_remove_stopwords(df_en['lyrics_formal'])

Processing: 100%|██████████| 3374198/3374198 [08:19<00:00, 6751.77it/s]


In [ ]:
df_en_cleaned_large = df_en.drop(columns=['lyrics_formal', 'language_cld3', 'language_ft'])

file_path = '/content/gdrive/MyDrive/bit_conference/coding/df_en_cleaned_large.csv'
df_en_cleaned_large.to_csv(file_path, index=False)
print(f"DataFrame successfully saved to {file_path}")

DataFrame successfully saved to /content/gdrive/MyDrive/bit_conference/coding/df_en_cleaned_large.csv
